# # Loading Data To Gold Zone 

**This Notebook:**
* Load data to Golg Zone of the Data Lake House
* Star Schekma and One Big Table Modeling
* Creates **`IDENTITY`** column in Databricks delta table

## 1.0 Initial Setup

In [0]:

%run "/Users/cabreirajm@gmail.com/DataPipelineCabreira/Helpers/data_generator" 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached dbldatagen-0.4.0.post1-py3-none-any.whl (122 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


## 2.0 Create `Gold Zone` Schema

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

DataFrame[]

## 3.0 `Sales Star Schema` Modeling 

Aiming to optimize queries in large datasets, we can use a dimensional model. 
We will use Ralph Kimball data warehouse principles and build a Star Schema model.


### `Dimensional Tables`
- **dim_calendar** - Dimension with date information
- **dim_cod** - Dimensions with codes  - Low cardinality Dimensions (Junk Dimension): 
  - **user_origin** - API vs. Files
  - **access_from** - mobile vs. computer
  - **payment_method** - Pix vs. Boleto vs. Cartão
  - **percent_discount** - 5% vs. 10% vs. 15%
- **dim_courses** - Dimensão responsável por armazenar as informações de Curso.
- **dim_user** - Dimensão responsável por armazenar as informações de Alunos.


All tables will have a **Surrogate Key (SK)** column that will be creeated with the **`<col_name> BIGINT GENERATED ALWAYS AS IDENTITY`** command. Spark will populate this column in execution time with an incremental value (incremental(1,1). )


### 3.1 `Sale Dimensions`  

In [0]:
#%sql
#DROP SCHEMA IF EXISTS gold CASCADE 


In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_calendar

res0: Boolean = true

In [0]:

%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_cod

res1: Boolean = true

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_course

res2: Boolean = true

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_user

res3: Boolean = true

In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_calendar(
        sk_tempo BIGINT GENERATED ALWAYS AS IDENTITY,
        date DATE,
        year INT, 
        month STRING,
        month_year INT,
        day_week_int INT, 
        day_week STRING,
        fl_day_week BOOLEAN,
        day_month INT,
        fl_last_month_day INT,
        day_year INT,
        week_year INT,
        bimonthly INT,
        quarter INT, 
        semester INT, 
        dt_load TIMESTAMP
    )  
""")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_cod (
        sk_cod BIGINT GENERATED ALWAYS AS IDENTITY,
        user_origin STRING,
        access_from STRING,
        payment_method STRING,
        percent_discount STRING,
        dt_load TIMESTAMP
    )  
""")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_course(
        sk_course BIGINT GENERATED ALWAYS AS IDENTITY,
        course_uuid STRING,
        course_name STRING, 
        course_level STRING,
        course_price DECIMAL(9,2),
        dt_load TIMESTAMP
    )          
""")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_user(
    sk_user BIGINT GENERATED ALWAYS AS IDENTITY,
    user_uuid STRING,
    user_name STRING,
    user_email STRING,
    user_idade INT, 
    user_gender STRING,
    user_state STRING,
    user_profession STRING,
    company STRING,
    dt_load TIMESTAMP
    )
""")

DataFrame[]

### 3.2 `Calendar Dimension`  

The view **`vw_dim_calendar`**:
* Starts date : **01/06/2024** 
* End date: **31/12/2025**

In [0]:

from pyspark.sql.functions import explode, sequence, to_date

start_date = '2024-06-01'
end_date = '2025-12-31'

spark.sql(f"""         
  with dates as (
    select
      explode(
        sequence(
          to_date('{start_date}'),
          to_date('{end_date}'),
          interval 1 day
        )
      ) as date
  )
  select
    date,
    year(date) AS year,
    to_csv(
      named_struct('date', date),
      map('dateFormat', 'MMMM', 'locale', 'EN')
    ) AS month,
    month(date) as month_year,
    dayofweek(date) AS day_week_int,
    to_csv(
      named_struct('date', date),
      map('dateFormat', 'EEEE', 'locale', 'EN')
    ) AS day_week,
    case
      when weekday(date) < 5 then True
      else False
    end as fl_day_week,
    dayofmonth(date) as day_month,
    case
      when date = last_day(date) then True
      else False
    end as fl_last_month_day,
    dayofyear(date) as day_year,
    weekofyear(date) as week_year,
    case
      when month(date) in (1, 2) then 1
      when month(date) in (3, 4) then 2
      when month(date) in (5, 6) then 3
      when month(date) in (7, 8) then 4
      when month(date) in (9, 10) then 5
      when month(date) in (11, 12) then 6
    end as bimonthly,
    case
      when month(date) in (1, 2, 3) then 1
      when month(date) in (4, 5, 6) then 2
      when month(date) in (7, 8, 9) then 3
      when month(date) in (10, 11, 12) then 4
    end as quarter,
    case
      when month(date) in (1, 2, 3, 4, 5, 6) then 1
      when month(date) in (7, 8, 9, 10, 11, 12) then 2
    end as semester
  from
    dates
""").createOrReplaceTempView('vw_dim_calendar')

spark.sql('SELECT * FROM vw_dim_calendar LIMIT 3').display()

date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester
2024-06-01,2024,June,6,7,Saturday,false,1,false,153,22,3,2,1
2024-06-02,2024,June,6,1,Sunday,false,2,false,154,22,3,2,1
2024-06-03,2024,June,6,2,Monday,true,3,false,155,23,3,2,1



We will now use the **`Merge`** commando to load the `dim_calendar` data.

In [0]:
spark.sql("""
    MERGE INTO gold.dim_calendar as dest
    USING vw_dim_calendar AS orig
        ON dest.date = orig.date
    WHEN NOT MATCHED    
        THEN INSERT(
            date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester,dt_load
            )
        VALUES(
            date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester,getdate()
        )
    """).display()


spark.sql("SELECT * FROM gold.dim_calendar LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
579,0,0,579


sk_tempo,date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester,dt_load
1,2024-06-01,2024,June,6,7,Saturday,false,1,0,153,22,3,2,1,2024-11-29T22:42:24.303Z
2,2024-06-02,2024,June,6,1,Sunday,false,2,0,154,22,3,2,1,2024-11-29T22:42:24.303Z
3,2024-06-03,2024,June,6,2,Monday,true,3,0,155,23,3,2,1,2024-11-29T22:42:24.303Z
4,2024-06-04,2024,June,6,3,Tuesday,true,4,0,156,23,3,2,1,2024-11-29T22:42:24.303Z
5,2024-06-05,2024,June,6,4,Wednesday,true,5,0,157,23,3,2,1,2024-11-29T22:42:24.303Z


## 3.3 Junk Dimension
We will first create a **`vw_dim_cod`** view that will be responsible for creating a **cartesian product between some of low cardinality codes***. 

We will use this method in order to avoid building low dimensions with few registers.


In [0]:
spark.sql( """
    SELECT DISTINCT 
    s.origin user_origin ,
    CASE WHEN s.origin = 'FILE' then "do not apply" ELSE a.local_access end as local_access,
    s.payment_method,
    coalesce(s.percent_discount, 'do not apply' ) as percent_discount
    from silver.tb_sales as s
    cross join silver.tb_access as a 
    order BY user_origin,local_access, payment_method, percent_discount      
""").createOrReplaceTempView("vw_dim_cod")

spark.sql("select * from vw_dim_cod limit 5").display()

user_origin,local_access,payment_method,percent_discount
API,Computer,boleto,5%
API,Computer,boleto,do not apply
API,Computer,credito,10%
API,Computer,credito,15%
API,Computer,credito,5%


In [0]:
spark.sql("""
    MERGE INTO gold.dim_cod as dest
    using vw_dim_cod as orig
    on dest.user_origin = orig.user_origin
        and dest.payment_method = orig.payment_method
        and dest.access_from = orig.local_access
        and dest.percent_discount = orig.percent_discount

    when not matched 
        then INSERT (
            user_origin,
            access_from,
            payment_method,
            percent_discount,
           dt_load
        )
        values(
            user_origin,
            local_access,
            payment_method,
            percent_discount,
           getdate()
        )       
""").display()

spark.sql("SELECT * FROM gold.dim_cod LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


sk_cod,user_origin,access_from,payment_method,percent_discount,dt_load
1,API,Computer,boleto,5%,2024-11-29T22:47:41.518Z
2,API,Computer,boleto,do not apply,2024-11-29T22:47:41.518Z
3,API,Computer,credito,10%,2024-11-29T22:47:41.518Z
4,API,Computer,credito,15%,2024-11-29T22:47:41.518Z
5,API,Computer,credito,5%,2024-11-29T22:47:41.518Z


## 3.4 User Dimension

In [0]:
spark.sql("""
      MERGE INTO gold.dim_user as dest 
      using silver.tb_users as orig
        on dest.user_uuid = orig.user_uuid
    when matched
        and dest.user_email != orig.user_email
        or dest.user_idade!= orig.user_idade
        or dest.user_state != orig.user_state
        or dest.user_profession != orig.user_profession
        or dest.company != orig.company
        THEN UPDATE 
            SET dest.user_email = orig.user_email
                ,dest.user_idade = orig.user_idade 
                ,dest.user_state = orig.user_state
                ,dest.user_profession = orig.user_profession
                ,dest.company  = orig.company
          
         when not MATCHED
            then insert (
        user_uuid, 
        user_name, 
        user_email, 
        user_idade, 
        user_gender,
        user_state, 
        user_profession, 
        company, 
        dt_load
            )
    VALUES (
        user_uuid, 
        user_name, 
        user_email, 
        user_idade, 
        user_gender,
        user_state, 
        user_profession, 
        company, 
        getdate()

            )   
""").display()

spark.sql("SELECT * FROM gold.dim_user LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
121227,0,0,121227


sk_user,user_uuid,user_name,user_email,user_idade,user_gender,user_state,user_profession,company,dt_load
1,7acec1578f16289a78314e9f67c600eb,Usuario 200a85dcac,usuario_200a85dcac@uol.com,33,M,PE,Analista de Negocio,null,2024-11-29T22:48:29.661Z
8,b82f38dbe5a5d8af7b4015fa99a4bff3,Usuario 0c7475d556,usuario_0c7475d556@outlook.com,31,F,CE,Arquiteto de Dados,null,2024-11-29T22:48:29.661Z
15,78b1d9f7cd2914aa507fd5d682828243,Usuario dd2496c049,usuario_dd2496c049@hotmail.com,46,F,SE,Desenvolvedor de ETL,null,2024-11-29T22:48:29.661Z
22,ead4be75d0192142a985c186477162b9,Usuario b3f6c8e7b5,usuario_b3f6c8e7b5@outlook.com,41,M,PB,Analista de Dados,null,2024-11-29T22:48:29.661Z
29,eab450bfd9dc75a88a587b0bae54842e,Usuario 8e2c3e7606,usuario_8e2c3e7606@hotmail.com,26,F,RO,Arquiteto de Dados,null,2024-11-29T22:48:29.661Z


## 3.5 Courses Dimension

We will load the **`dim_course`** with  **`MERGE`**  command
* Source: **`silver.tb_curso`**. table


In [0]:
spark.sql("""
    merge into gold.dim_course as dest 
        using silver.tb_courses as orig
            on dest.course_uuid = orig.course_uuid

    when matched    
        and dest.course_name != orig.course_name
        or dest.course_level != orig.course_level
        or dest.course_price != orig.course_price
        then update 
            set 
                dest.course_name = orig.course_name,
                dest.course_level = orig.course_level,
                dest.course_price = orig.course_price
    when not matched
        then insert (
            course_uuid,
            course_name,
            course_level,
            course_price,
            dt_load
        )
    values (
            course_uuid,
            course_name,
            course_level,
            course_price,
            GETDATE()
    )
""").display()

spark.sql("SELECT * FROM gold.dim_course LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,0,0,3


sk_course,course_uuid,course_name,course_level,course_price,dt_load
1,f260cd97c6c9813b01601e834a2added,Construindo o seu Primeiro Pipeline de Dados com o Databricks,beginner,589.90,2024-11-29T22:48:39.282Z
2,34bdd77f6954552d11c4f5547cb41458,Do Primeiro Pipeline ao Data Lakehouse com o Databricks,intermediate,659.90,2024-11-29T22:48:39.282Z
3,c2d6bcbc3e46555bb1e7e9afbc24d3af,Construindo Pipelines de Dados usando o Spark Structured Streaming,advanced,549.90,2024-11-29T22:48:39.282Z


## 3.6 Sales Fact

In [0]:
df_stream_sales= spark.readStream.table('silver.tb_sales')
df_stream_sales.createOrReplaceTempView('vw_fact_sales')

In [0]:
df_fact_sales = spark.sql("""
        with orig_fact_sales (
        select 
            s.percent_discount,
            case when s.origin = 'FILE' then 'not applicable' else a.local_access end as access_local,
            s.payment_method,
            s.origin as user_origin,
            s.discount_value,
            s.total_value,
            s.course_uuid,
            s.user_uuid,
            s.dt_sale
        from vw_fact_sales as s
        left join silver.tb_access as a on s.access_uuid = a.access_uuid
    ) select 
        dcal.sk_tempo,
        dcod.sk_cod,
        dcod.user_origin,
        dcod.payment_method,
        dcod.access_from,
        dc.sk_course,
        du.sk_user,
        ofs.access_local,
        ofs.discount_value,
        ofs.total_value
    from orig_fact_sales as ofs
    join gold.dim_cod as dcod on ofs.payment_method = dcod.payment_method
                                and ofs.user_origin = dcod.user_origin
                               and ofs.access_local =dcod.access_from
                               and ofs.percent_discount = dcod.percent_discount
    join gold.dim_calendar as dcal on dcal.date = cast(ofs.dt_sale as date)
    join gold.dim_course as dc on dc.course_uuid = ofs.course_uuid
    join gold.dim_user as du on du.user_uuid = ofs.user_uuid
    
""")

df_fact_sales.limit(5).display()

sk_tempo,sk_cod,user_origin,payment_method,access_from,sk_course,sk_user,access_local,discount_value,total_value
1,3,API,credito,Computer,2,169,Computer,65.99,593.90
1,3,API,credito,Computer,2,169,Computer,65.99,593.90
2,1,API,boleto,Computer,3,190,Computer,27.50,522.40
2,1,API,boleto,Computer,3,190,Computer,27.50,522.40
2,5,API,credito,Computer,3,253,Computer,27.50,522.40


In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/_checkpoint/fact_sales


res4: Boolean = true

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/f_sales

res5: Boolean = true

In [0]:
sales_fact_gold_checkpoint_path = 'dbfs:/user/hive/warehouse/gold.db/_checkpoint/fact_sales'
(

df_fact_sales.writeStream
            .format('delta')
            .outputMode('append')
            .option('checkpointLocation', sales_fact_gold_checkpoint_path)
            .trigger(availableNow=True)
            .toTable('gold.f_sales')
).awaitTermination()

spark.sql('select * from gold.f_sales limit 5').display()

sk_tempo,sk_cod,user_origin,payment_method,access_from,sk_course,sk_user,access_local,discount_value,total_value
1,3,API,credito,Computer,2,169,Computer,65.99,593.90
1,3,API,credito,Computer,2,169,Computer,65.99,593.90
2,1,API,boleto,Computer,3,190,Computer,27.50,522.40
2,1,API,boleto,Computer,3,190,Computer,27.50,522.40
2,5,API,credito,Computer,3,253,Computer,27.50,522.40


* The **`.writeStream`** method will be used to store data from **`df_fact_sales`** dataframe into the **gold.f_sales**. This table will be created on-the-fly trough the **`toTable()`** method.

* The **`outputMode('Append')`** states that the data will be appended in the destination table

* **`option('checkpointLocation', sales_fact_gold_checkpoint_path)`** defines the checkpoint directory that spark will use to control the stream processing in order to do the `exactly-once delivery` during the load processing.

*  **`.trigger(availableNow=True)`** indicates to writeStream method that **the process will occour in batch**

* **`.awaitTermination()`** is used to make the stream query a synchronous process.

The code below simulates a **sales analysis report**. This report shows the revenue + qnt of sold courses in jun24 with 5% discount applied and that were done by mobile.


In [0]:
spark.sql("""
    select 
        count(*) as qnt_sold_courses,
        sum(f.total_value) - sum(f.discount_value) as total_revenue
    from gold.f_sales as f
    join gold.dim_cod as dcod 
    on f.sk_cod = dcod.sk_cod
    join gold.dim_course as dco
    on dco.sk_course = f.sk_course
    join gold.dim_calendar dca
    on dca.sk_tempo = f.sk_tempo
    join gold.dim_user du
    on du.sk_user = f.sk_user
    where 
    dca.year = 2024
    and dca.month = 'June'
    and dcod.access_from = 'Mobile'
    and dcod.percent_discount = '5%'

""").display()

qnt_sold_courses,total_revenue
250,134389.91


## 4.0 One Big Table (OBT)

**One Big Table (OBT)** is an alternative to use stead of dimensional modeling. This data modeling is **Simple** and **stores all the data in one unique table** (big table). This offers a simplified model by reducing the number of tables that need to be updated and governated.


In [0]:
df_stream_acesse = spark.readStream.table('silver.tb_access')
df_stream_acesse.createOrReplaceTempView('vw_access')

Lets create the **`df_obt_acesso`** dataframe as result of a query that performs a join between the **`vw_access`** stream view created above and the  **`gold.dim_tempo`** table to get tww columns.


In [0]:
%sql
select * from silver.tb_sales limit 3

dt_sale,access_uuid,user_uuid,course_uuid,payment_method,qnt_instalments,instalments_values,total_value,percent_discount,discount_value,origin,dt_load
2024-06-02T22:55:48Z,6706860c19432db16b78d9c1b9a02906,b82f38dbe5a5d8af7b4015fa99a4bff3,f260cd97c6c9813b01601e834a2added,credito,12,49.16,589.92,null,null,API,2024-11-29T22:38:53.749Z
2024-06-03T14:17:56Z,cc811bb776e337c959807b9020cef6c6,d47f5c973872bf06a887add70d895aa1,c2d6bcbc3e46555bb1e7e9afbc24d3af,boleto,1,549.90,549.90,null,null,API,2024-11-29T22:38:53.749Z
2024-06-03T15:56:44Z,a1b085b6325000741ff5c011cbe7ebc1,cff7983c2a789adbe0628a74c4d1004f,c2d6bcbc3e46555bb1e7e9afbc24d3af,credito,2,274.95,549.90,null,null,API,2024-11-29T22:38:53.749Z


In [0]:
df_obt_access = spark.sql("""
    select 
        a.local_access,
        a.access_ip_address,
        c.date,
        c.year,
        c.month,
        c.month_year,
        c.day_month,
        c.day_week,
        c.day_week_int,
        c.fl_day_week,
        c.day_year,
        c.fl_last_month_day,
        c.week_year,
        c.semester,
        c.quarter,
        c.bimonthly,
        1 as qnt_access,        
        CASE WHEN a.user_uuid IS NOT NULL THEN 1 ELSE 0 END AS qnt_user,
        CASE WHEN s.qnt_instalments IS NOT NULL THEN 1 ELSE 0 END AS qnt_sales
        from vw_access AS a
        join gold.dim_calendar as c on cast(a.access_timestamp as date) = c.date 
        left join silver.tb_sales s on a.access_uuid = s.access_uuid

""")

df_obt_access.limit(5).display()

local_access,access_ip_address,date,year,month,month_year,day_month,day_week,day_week_int,fl_day_week,day_year,fl_last_month_day,week_year,semester,quarter,bimonthly,qnt_access,qnt_user,qnt_sales
Computer,69.127.75.83,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0
Computer,168.18.37.100,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0
Mobile,113.109.66.208,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,1,0
Computer,87.241.252.59,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0
Mobile,188.111.120.11,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0


* **`.writeStream`** : Writes  **`df_obt_access`** data into **gold.obt_access** table that will be created on-the-fly through the method  **`table`**  

* **`.outputMode('append')`** : States append mode for data storage 

* .option('CheckpointLocation', obt_gold_checkpoint_path) : Identify the checkpoint directory that spark will use to control the stream data and perform `**exaclty-once delivery**`

* **`.trigger(availableNow=True)`** : Indicates to writeStream that the process will occour in batch


* **`.awaitTermination()`**: Makes the query an asynchronous process

In [0]:
obt_gold_checkpoint_path = 'dbfs:/user/hive/warehouse/gold.db/_checkpoint/obt_acesso'
(
df_obt_access.writeStream
        .format('delta')
        .outputMode('append')
        .option('CheckpointLocation', obt_gold_checkpoint_path)
        .option('mergeSchema',True)
        .trigger(availableNow=True)
        .table('gold.obt_access')
).awaitTermination()

spark.sql('select * from gold.obt_access limit 5').display()

local_access,access_ip_address,date,year,month,month_year,day_month,day_week,day_week_int,fl_day_week,day_year,fl_last_month_day,week_year,semester,quarter,bimonthly,qnt_access,qnt_user,qnt_sales
Computer,69.127.75.83,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0
Computer,168.18.37.100,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0
Mobile,113.109.66.208,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,1,0
Computer,87.241.252.59,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0
Mobile,188.111.120.11,2024-06-02,2024,June,6,2,Sunday,1,false,154,0,22,1,2,3,1,0,0


In [0]:
%sql 
SELECT 
  SUM(qnt_access) AS qnt_access,
  SUM(qnt_user) AS qt_users,
  SUM(qnt_sales) AS qnt_sales,
  CAST((SUM(qnt_sales)/SUM(qnt_access))*100 AS INT) AS percent_sales
FROM gold.obt_access
WHERE local_access = 'Computer'
  AND month = 'June'
  AND year = 2024
  AND fl_day_week is false

qnt_access,qt_users,qnt_sales,percent_sales
29938,8964,3178,10


In [0]:
stop_all_streams()